In [1]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras import optimizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

Using TensorFlow backend.


In [2]:
list_filename="gspy-db-20180813_O1_filtered_t1126400691-1205493119_snr7.5_tr_gspy.csv"

In [3]:
data_dir = os.path.join(os.path.dirname(os.getcwd()),"data")
data_dir

'/notebooks/data'

In [4]:
gl_df = pd.read_csv(os.path.join(data_dir,list_filename))
gl_df.describe()

,GPStime,peakFreq,snr,centralFreq,duration,bandwidth
count,6.667000e+03,6667.000000,6667.000000,6667.000000,6667.000000,6667.000000
mean,1.131870e+09,204.124335,192.144943,1529.353582,1.779605,2941.086955
std,3.165652e+06,374.454670,1589.944713,1320.377343,2.672461,2664.103210
min,1.126403e+09,10.072000,7.501000,9.780000,0.007000,1.259180
25%,1.128905e+09,34.175000,10.337000,255.412000,0.227000,423.261902
50%,1.132168e+09,111.128000,15.407000,1228.915000,0.766000,2320.672363
75%,1.134571e+09,183.495000,36.997500,2630.882000,2.144500,5228.188477
max,1.137250e+09,2047.106000,81178.727000,4615.132000,42.156000,7946.482910


In [5]:
gl_df.shape

(6667, 9)

In [8]:
gl_df.tail()

,GPStime,peakFreq,snr,centralFreq,duration,bandwidth,id,ifo,label
6662,1.129802e+09,2025.310,27.166,1838.466,0.971,3612.932861,otpK5QYIAh,H1,Extremely_Loud
6663,1.130646e+09,10.416,21.426,3876.353,4.750,7742.791016,9iEHyEEJZx,H1,Low_Frequency_Lines
6664,1.127385e+09,48.344,7.698,776.262,0.577,1457.913940,OonOZHaR0U,H1,None_of_the_Above
6665,1.132625e+09,27.900,27.530,3881.117,2.841,7733.262695,ijzVn9vaOe,L1,No_Glitch
6666,1.131374e+09,18.986,17.409,2960.423,3.875,5903.526855,8MiALG6Akm,H1,Light_Modulation


In [9]:
X = gl_df.get(['GPStime','peakFreq','snr','centralFreq','duration','bandwidth','id','ifo'])
Y = gl_df.get('label')

print('X : ')
print(X.count())
print()
print('Y: ')
print(Y.count())

Y_num = pd.factorize(Y)
Y_num_labels = Y_num[0]
Y_count_labels = Y_num[1].size

print()
print('Distinct classes :', Y_count_labels)
print()
print('Y_num_labels : ', Y_num_labels.size)

X : 
GPStime        6667
peakFreq       6667
snr            6667
centralFreq    6667
duration       6667
bandwidth      6667
id             6667
ifo            6667
dtype: int64

Y: 
6667

Distinct classes : 22

Y_num_labels :  6667


In [19]:
gl_df['ifo'].unique()

array(['H1', 'L1'], dtype=object)

In [10]:
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_num_labels, test_size=test_size, random_state=seed)

In [11]:
print('X_train : ')
print(X_train.count())
print()
print('Y_train: ')
print(Y_train.size)
print()
print('X_test: ')
print(X_test.count())
print()
print('Y_test: ')
print(Y_test.size)

X_train : 
GPStime        4466
peakFreq       4466
snr            4466
centralFreq    4466
duration       4466
bandwidth      4466
id             4466
ifo            4466
dtype: int64

Y_train: 
4466

X_test: 
GPStime        2201
peakFreq       2201
snr            2201
centralFreq    2201
duration       2201
bandwidth      2201
id             2201
ifo            2201
dtype: int64

Y_test: 
2201


In [ ]:
training = xgb.DMatrix(X_train, label=Y_train)
test = xgb.DMatrix(X_test, label=Y_test)

num_round = 2
param = {'max_depth': 2, 
         'eta': 1, 
         'silent':1, 
         'objective':'multi:softmax',
         'num_class' : Y_count_labels}
bst = xgb.train(param, training, num_round)
preds = bst.predict(test)

In [131]:
error_rate = np.sum(preds != Y_test) / Y_test.size
error_rate

0.17587716044803797

In [542]:
print(classification_report(Y_test, preds))

              precision    recall  f1-score   support

           0       0.82      0.79      0.81       231
           1       0.69      0.65      0.67      1265
           2       0.92      0.78      0.85      4285
           3       0.40      0.37      0.38       555
           4       0.81      0.46      0.59      1277
           5       0.61      0.84      0.71      1277
           6       0.86      0.61      0.71       945
           7       0.99      1.00      0.99     10032
           8       0.75      0.66      0.70       825
           9       0.91      0.88      0.89        56
          10       0.63      0.25      0.35       130
          11       0.33      0.18      0.24        33
          12       0.89      0.85      0.87       878
          13       0.82      0.85      0.83      2730
          14       0.77      0.70      0.73      1909
          15       0.06      0.34      0.10       235
          16       0.78      0.92      0.84       276
          17       0.21    

/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
